In [116]:
import pandas as pd
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import logging
import numpy as np
import talib

logging.basicConfig(level=logging.INFO)

In [117]:
INTERVAL_1_MIN = pd.Timedelta(minutes=1)
INTERVAL_5_MIN = pd.Timedelta(minutes=5)
INTERVAL_15_MIN = pd.Timedelta(minutes=15)
INTERVAL_30_MIN = pd.Timedelta(minutes=30)


In [118]:
merdeg_data_path = "/teamspace/studios/this_studio/informer-validation/data/btc_with_macro.csv"

merged_df = pd.read_csv(merdeg_data_path, parse_dates=["datetime"])

# Hourly price data
df_5min  = merged_df.resample('5min', on='datetime').first().reset_index()


df_5min

,datetime,close,high,low,open,volume,fng_value,vix_value,fed_funds_rate
0,2023-01-01 00:00:00,16543.67,16544.76,16538.45,16541.77,83.08143,26.0,21.67,4.33
1,2023-01-01 00:05:00,16536.70,16537.80,16533.94,16534.91,53.58957,26.0,21.67,4.33
2,2023-01-01 00:10:00,16525.34,16530.87,16522.55,16526.67,96.60421,26.0,21.67,4.33
3,2023-01-01 00:15:00,16521.00,16526.84,16519.40,16521.26,114.31289,26.0,21.67,4.33
4,2023-01-01 00:20:00,16536.61,16537.82,16534.15,16534.94,37.63557,26.0,21.67,4.33
...,...,...,...,...,...,...,...,...,...
230683,2025-03-11 23:35:00,82699.67,82699.93,82622.63,82627.82,4.78086,24.0,26.92,4.33
230684,2025-03-11 23:40:00,82731.14,82884.81,82731.14,82880.90,19.86220,24.0,26.92,4.33
230685,2025-03-11 23:45:00,82818.44,82818.45,82732.01,82786.94,11.97104,24.0,26.92,4.33
230686,2025-03-11 23:50:00,82699.99,82700.00,82663.93,82663.93,2.94964,24.0,26.92,4.33


In [119]:
def preprocess_augment_data(data, interval):
    assert 'close' in data.columns
    assert 'datetime' in data.columns

    result = data.copy()
    
     # ensure datetime index
    result["datetime"] = pd.to_datetime(result["datetime"])
    result = result.set_index("datetime").sort_index()
    result = result.interpolate(method="time")
    result['datetime'] = result.index

    # Required for pytorch forecasting framework
    result['group_id'] = 'BTCUSDT'

    # Derrived variables
    result['hour'] = result['datetime'].apply(lambda x: x.hour).astype('str')
    result['weekday'] = result['datetime'].apply(lambda x: x.weekday()).astype('str')
    result['open_to_close'] = result['open'] / result['close']
    result['high_to_close'] = result['high'] / result['close']
    result['low_to_close'] = result['low'] / result['close']
    result['high_to_low_price'] = result['high'] / result['low']

    # Resturns
    ret = result['close'].to_numpy()
    ret = (ret[1:] / ret[:-1]) - 1
    result['returns'] = np.pad(ret, (1, 0), 'constant', constant_values=(0, 0))

    # Returns binary
    result['returns_binary'] = (result['returns'] > 0).astype(np.int32)

    # Log returns
    log_ret = result['close'].to_numpy()
    log_ret = np.log(log_ret[1:]) - np.log(log_ret[:-1])
    result['log_returns'] = np.pad(log_ret, (1, 0), 'constant', constant_values=(0, 0))

    # ::Realized volatility
    cum_sum = result['log_returns'].to_numpy()
    cum_sum *= cum_sum
    cum_sum = np.cumsum(cum_sum)
    
    close = result['close'].to_numpy()
    # 1h window 
    obs_1h = 1 * (60 // interval)
    result['vol_1h'] = np.sqrt(cum_sum - np.pad(cum_sum[:-obs_1h], (obs_1h, 0), 'constant', constant_values=(0, 0)))
    result['sma_1h_to_close'] = (talib.SMA(close, obs_1h) / result['close']).fillna(0)
    result['ema_1h_to_close'] = (talib.EMA(close, obs_1h) / result['close']).fillna(0)
    # 1d window
    obs_1d = 24 * (60 // interval)
    result['vol_1d'] = np.sqrt(cum_sum - np.pad(cum_sum[:-obs_1d], (obs_1d, 0), 'constant', constant_values=(0, 0)))
    result['sma_1d_to_close'] = (talib.SMA(close, obs_1d) / result['close']).fillna(0)
    result['ema_1d_to_close'] = (talib.EMA(close, obs_1d) / result['close']).fillna(0)
    # 7d window
    obs_7d = 7 * 24 * (60 // interval)
    result['vol_7d'] = np.sqrt(cum_sum - np.pad(cum_sum[:-obs_7d], (obs_7d, 0), 'constant', constant_values=(0, 0)))
    result['sma_7d_to_close'] = (talib.SMA(close, obs_7d) / result['close']).fillna(0)

    macd, signal, _ = talib.MACD(close, fastperiod=12, slowperiod=26, signalperiod=9)
    result['macd'] = macd
    result['macd'] = result['macd'].fillna(0)
    result['macd_signal'] = signal
    result['macd_signal'] = result['macd_signal'].fillna(0)

    rsi = talib.RSI(close, timeperiod=14)
    result['rsi'] = rsi
    result['rsi'] = result['rsi'].fillna(0)

    upper, middle, lower = talib.BBANDS(close, 20, 2.0, 2.0)
    result['low_bband_to_close'] = (lower / result['close']).fillna(0)
    result['up_bband_to_close'] = (upper / result['close']).fillna(0)
    result['mid_bband_to_close'] = (middle / result['close']).fillna(0)

    result = result.dropna(axis=0)
    result['time_index'] = range(len(result))
    
    assert len(result.dropna()) == len(result) 
    assert len(result['time_index'].unique()) == len(result)

    return result


df_feat = preprocess_augment_data(df_5min, 5)

df_feat.to_csv("/teamspace/studios/this_studio/informer-validation/data/btcusdt_5min_features.csv", index=False)
df_feat

,close,high,low,open,volume,fng_value,vix_value,fed_funds_rate,datetime,group_id,...,ema_1d_to_close,vol_7d,sma_7d_to_close,macd,macd_signal,rsi,low_bband_to_close,up_bband_to_close,mid_bband_to_close,time_index
datetime,,,,,,,,,,,,,,,,,,,,,
2023-01-01 00:00:00,16543.67,16544.76,16538.45,16541.77,83.08143,26.0,21.67,4.33,2023-01-01 00:00:00,BTCUSDT,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0
2023-01-01 00:05:00,16536.70,16537.80,16533.94,16534.91,53.58957,26.0,21.67,4.33,2023-01-01 00:05:00,BTCUSDT,...,0.000000,0.000421,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1
2023-01-01 00:10:00,16525.34,16530.87,16522.55,16526.67,96.60421,26.0,21.67,4.33,2023-01-01 00:10:00,BTCUSDT,...,0.000000,0.000806,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2
2023-01-01 00:15:00,16521.00,16526.84,16519.40,16521.26,114.31289,26.0,21.67,4.33,2023-01-01 00:15:00,BTCUSDT,...,0.000000,0.000848,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3
2023-01-01 00:20:00,16536.61,16537.82,16534.15,16534.94,37.63557,26.0,21.67,4.33,2023-01-01 00:20:00,BTCUSDT,...,0.000000,0.001269,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-03-11 23:35:00,82699.67,82699.93,82622.63,82627.82,4.78086,24.0,26.92,4.33,2025-03-11 23:35:00,BTCUSDT,...,0.986585,0.109223,1.036449,-85.039185,-30.226108,39.582644,0.998928,1.008128,1.003528,230683
2025-03-11 23:40:00,82731.14,82884.81,82731.14,82880.90,19.86220,24.0,26.92,4.33,2025-03-11 23:40:00,BTCUSDT,...,0.986305,0.109223,1.036027,-87.898704,-41.760627,41.241269,0.998214,1.007523,1.002869,230684
2025-03-11 23:45:00,82818.44,82818.45,82732.01,82786.94,11.97104,24.0,26.92,4.33,2025-03-11 23:45:00,BTCUSDT,...,0.985368,0.109228,1.034907,-82.173270,-49.843156,45.695041,0.996983,1.006323,1.001653,230685
